# Query an existing Chroma DB with GUI (Gradio)
Sample of how to querty a Chroma Vector Database oriented to creating a RAG system.

In [ ]:
# https://python.langchain.com/docs/integrations/vectorstores/chroma/

In [ ]:
# !pip install sentence-transformers

In [ ]:
# !pip install gradio

Select **model** for embeddings. 

We must to select the same model of existing embeddings in database

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Access existing database

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="some_facts",
    embedding_function=embeddings,
    persist_directory="./chroma_some_facts",  # Where to save data locally, remove if not necessary
)

## Searching

In [ ]:
# you don't need to run this section

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content}")

In [ ]:
# we can configure a 'RETRIEVER', a key component in Langchain used to find relevant information from document collections

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)
retriever.invoke("Stealing from the bank is a crime")

## Create a Gradio interface

In [ ]:
# Create a function to make the query to the database 
# We use almost the same code that previusly, but in function style ;-)

In [ ]:
def search_chroma(query, top_k):
    try:   
        results = vector_store.similarity_search(query, k=top_k)
        return "\n\n".join(
            [f"**Result {i+1}:**\n{doc.page_content}" for i, doc in enumerate(results)]
        )
    except Exception as e:
        return f"Error: {e}"
    

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### Chroma Database Search")
    
    with gr.Row():
        query_input = gr.Textbox(label="Enter Your Query", placeholder="Type your question here...")
        top_k_input = gr.Slider(1, 10, step=1, value=5, label="Number of Results")

    search_button = gr.Button("Search")
    output_box = gr.Textbox(label="Search Results", lines=15)

    # Bind the function to the Gradio UI
    search_button.click(fn=search_chroma, inputs=[query_input, top_k_input], outputs=output_box)

# Launch the App
demo.launch()

In [ ]:
# We can do it with a retriever

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)

def search_with_retriever(query, top_k):
    try:
        retriever.search_kwargs["k"] = top_k  # Dynamically set the number of results
        results = retriever.get_relevant_documents(query)
        return "\n\n".join(
            [f"**Result {i+1}:**\n{doc.page_content}" for i, doc in enumerate(results)]
        )
    except Exception as e:
        return f"Error: {e}"

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### Chroma Database Search")
    
    with gr.Row():
        query_input = gr.Textbox(label="Enter Your Query", placeholder="Type your question here...")
        top_k_input = gr.Slider(1, 10, step=1, value=5, label="Number of Results")

    search_button = gr.Button("Search")
    output_box = gr.Textbox(label="Search Results", lines=15)

    # Bind the function to the Gradio UI
    search_button.click(fn=search_with_retriever, inputs=[query_input, top_k_input], outputs=output_box)

# Launch the App
demo.launch()